In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except:
    XGBOOST_AVAILABLE = False

try:
    from lightgbm import LGBMClassifier
    LIGHTGBM_AVAILABLE = True
except:
    LIGHTGBM_AVAILABLE = False

In [ ]:
df = pd.read_csv('credit_risk_dataset.csv')
print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()

print("Dataset Info:")
print(df.info())
print("\n" + "="*60)
print("\nMissing Values:")
print(df.isnull().sum())
print("\n" + "="*60)
print("\nStatistical Summary:")
df.describe()

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"Numeric columns: {numeric_cols}")
print(f"\nCategorical columns: {categorical_cols}")

if 'loan_status' in df.columns:
    target_col = 'loan_status'
elif 'default' in df.columns:
    target_col = 'default'
elif 'target' in df.columns:
    target_col = 'target'
else:
    target_col = df.columns[-1]

print(f"\nTarget column: {target_col}")
print(f"\nTarget distribution:\n{df[target_col].value_counts()}")
print(f"\nTarget distribution (%):\n{df[target_col].value_counts(normalize=True) * 100}")

In [ ]:
plt.figure(figsize=(10, 6))
df[target_col].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Target Variable Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Class', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
if len(numeric_cols) > 0:
    n_cols = min(3, len(numeric_cols))
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for idx, col in enumerate(numeric_cols):
        if idx < len(axes):
            axes[idx].hist(df[col].dropna(), bins=30, color='steelblue', edgecolor='black', alpha=0.7)
            axes[idx].set_title(f'Distribution of {col}', fontsize=11, fontweight='bold')
            axes[idx].set_xlabel(col, fontsize=10)
            axes[idx].set_ylabel('Frequency', fontsize=10)
            axes[idx].grid(alpha=0.3)
    
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Distribution of Numeric Features', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

if len(numeric_cols) > 1:
    plt.figure(figsize=(14, 10))
    correlation_matrix = df[numeric_cols].corr()
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Matrix of Numeric Features', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()

In [ ]:
if len(categorical_cols) > 0:
    n_cols = min(3, len(categorical_cols))
    n_rows = (len(categorical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for idx, col in enumerate(categorical_cols):
        if idx < len(axes):
            value_counts = df[col].value_counts()
            axes[idx].bar(range(len(value_counts)), value_counts.values, color='coral', edgecolor='black', alpha=0.7)
            axes[idx].set_title(f'Distribution of {col}', fontsize=11, fontweight='bold')
            axes[idx].set_xlabel(col, fontsize=10)
            axes[idx].set_ylabel('Count', fontsize=10)
            axes[idx].set_xticks(range(len(value_counts)))
            axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
            axes[idx].grid(axis='y', alpha=0.3)
    
    for idx in range(len(categorical_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Distribution of Categorical Features', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
df_processed = df.copy()

for col in numeric_cols:
    if df_processed[col].isnull().sum() > 0:
        df_processed[col].fillna(df_processed[col].median(), inplace=True)

for col in categorical_cols:
    if df_processed[col].isnull().sum() > 0:
        df_processed[col].fillna(df_processed[col].mode()[0], inplace=True)

print(f"Missing values after handling: {df_processed.isnull().sum().sum()}")

In [ ]:
label_encoders = {}
for col in categorical_cols:
    if col != target_col:
        le = LabelEncoder()
        df_processed[col] = le.fit_transform(df_processed[col].astype(str))
        label_encoders[col] = le

if df_processed[target_col].dtype == 'object':
    le_target = LabelEncoder()
    df_processed[target_col] = le_target.fit_transform(df_processed[target_col])

print("Categorical encoding completed")

In [ ]:
X = df_processed.drop(columns=[target_col])
y = df_processed[target_col]

for col in X.select_dtypes(include=[np.number]).columns:
    Q1 = X[col].quantile(0.25)
    Q3 = X[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    X[col] = np.clip(X[col], lower_bound, upper_bound)

print("Outlier treatment completed")

In [ ]:
numeric_features = X.select_dtypes(include=[np.number]).columns

if 'loan_amnt' in numeric_features:
    X['loan_amnt_log'] = np.log1p(X['loan_amnt'])
if 'annual_inc' in numeric_features:
    X['annual_inc_log'] = np.log1p(X['annual_inc'])
if 'loan_amnt' in numeric_features and 'annual_inc' in numeric_features:
    X['debt_to_income'] = X['loan_amnt'] / (X['annual_inc'] + 1)
if 'loan_amnt' in numeric_features and 'int_rate' in numeric_features:
    X['total_interest'] = X['loan_amnt'] * X['int_rate'] / 100

for col in numeric_features:
    if X[col].std() > 0:
        X[f'{col}_squared'] = X[col] ** 2
        X[f'{col}_sqrt'] = np.sqrt(np.abs(X[col]))

print(f"Feature engineering completed. New shape: {X.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, n_jobs=-1),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'SVM': SVC(probability=True, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB()
}

if XGBOOST_AVAILABLE:
    models['XGBoost'] = xgb.XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1)

if LIGHTGBM_AVAILABLE:
    models['LightGBM'] = LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1)

print(f"Initialized {len(models)} models")

results = {}
trained_models = {}

for name, model in models.items():
    if name in ['SVM', 'KNN']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    try:
        roc_auc = roc_auc_score(y_test, y_pred_proba)
    except:
        roc_auc = 0.0
    
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    }
    
    trained_models[name] = model
    print(f"{name:25s} - Accuracy: {accuracy:.4f}")

print("\nModel training completed!")

In [ ]:
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[m]['Accuracy'] for m in results.keys()],
    'Precision': [results[m]['Precision'] for m in results.keys()],
    'Recall': [results[m]['Recall'] for m in results.keys()],
    'F1-Score': [results[m]['F1-Score'] for m in results.keys()],
    'ROC-AUC': [results[m]['ROC-AUC'] for m in results.keys()]
})

comparison_df = comparison_df.sort_values('Accuracy', ascending=False).reset_index(drop=True)

print("="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

In [ ]:
accuracy_differences = {}
base_accuracy = comparison_df.iloc[0]['Accuracy']
base_model = comparison_df.iloc[0]['Model']

for idx, row in comparison_df.iterrows():
    diff = base_accuracy - row['Accuracy']
    accuracy_differences[row['Model']] = diff
    print(f"{row['Model']:25s} - Accuracy: {row['Accuracy']:.4f} | Difference from {base_model}: {diff:.4f}")

print(f"\nBest Model: {base_model} with Accuracy: {base_accuracy:.4f}")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = plt.cm.Set3(np.linspace(0, 1, len(comparison_df)))

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    bars = ax.barh(comparison_df['Model'], comparison_df[metric], color=colors)
    ax.set_xlabel(metric, fontsize=12)
    ax.set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    ax.set_xlim(0, 1)
    
    for i, (bar, val) in enumerate(zip(bars, comparison_df[metric])):
        ax.text(val + 0.01, i, f'{val:.4f}', va='center', fontsize=9)
    
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.suptitle('Model Performance Metrics Comparison', fontsize=16, fontweight='bold', y=1.02)
plt.show()

In [ ]:
plt.figure(figsize=(12, 7))
bars = plt.barh(comparison_df['Model'], comparison_df['Accuracy'], color=colors)
plt.xlabel('Accuracy Score', fontsize=12)
plt.title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
plt.xlim(0, 1)

for bar, val in zip(bars, comparison_df['Accuracy']):
    plt.text(val + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{val:.4f}', va='center', fontsize=10, fontweight='bold')

plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
metrics_data = comparison_df.set_index('Model')[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']]
sns.heatmap(metrics_data, annot=True, fmt='.4f', cmap='YlOrRd', 
            cbar_kws={'label': 'Score'}, linewidths=0.5)
plt.title('Model Performance Heatmap', fontsize=14, fontweight='bold', pad=20)
plt.ylabel('Model', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve

plt.figure(figsize=(12, 8))

for name in results.keys():
    if name in ['SVM', 'KNN']:
        y_pred_proba = trained_models[name].predict_proba(X_test_scaled)[:, 1]
    else:
        y_pred_proba = trained_models[name].predict_proba(X_test)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = results[name]['ROC-AUC']
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.4f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - All Models Comparison', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
n_models = len(results)
n_cols = 3
n_rows = (n_models + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
axes = axes.flatten() if n_models > 1 else [axes]

for idx, (name, result) in enumerate(results.items()):
    if name in ['SVM', 'KNN']:
        y_pred = trained_models[name].predict(X_test_scaled)
    else:
        y_pred = trained_models[name].predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                cbar_kws={'label': 'Count'}, linewidths=0.5)
    axes[idx].set_title(f'{name}\nAccuracy: {result["Accuracy"]:.4f}', 
                        fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Predicted', fontsize=10)
    axes[idx].set_ylabel('Actual', fontsize=10)

for idx in range(n_models, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Confusion Matrices - All Models', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

cv_results = {}

for name, model in trained_models.items():
    if name in ['SVM', 'KNN']:
        cv_scores = cross_val_score(model, X_train_scaled, y_train, 
                                    cv=5, scoring='accuracy', n_jobs=-1)
    else:
        cv_scores = cross_val_score(model, X_train, y_train, 
                                    cv=5, scoring='accuracy', n_jobs=-1)
    
    cv_results[name] = {
        'Mean CV Score': cv_scores.mean(),
        'Std CV Score': cv_scores.std()
    }
    print(f"{name:25s} - CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

cv_df = pd.DataFrame({
    'Model': list(cv_results.keys()),
    'Mean CV Accuracy': [cv_results[m]['Mean CV Score'] for m in cv_results.keys()],
    'Std CV Accuracy': [cv_results[m]['Std CV Score'] for m in cv_results.keys()]
}).sort_values('Mean CV Accuracy', ascending=False).reset_index(drop=True)

print("\n" + "="*70)
print("CROSS-VALIDATION RESULTS")
print("="*70)
print(cv_df.to_string(index=False))

In [ ]:
final_comparison = comparison_df.merge(cv_df, on='Model')
final_comparison['Overall Score'] = (
    final_comparison['Accuracy'] * 0.25 +
    final_comparison['Precision'] * 0.20 +
    final_comparison['Recall'] * 0.20 +
    final_comparison['F1-Score'] * 0.20 +
    final_comparison['ROC-AUC'] * 0.10 +
    final_comparison['Mean CV Accuracy'] * 0.05
)
final_comparison = final_comparison.sort_values('Overall Score', ascending=False).reset_index(drop=True)

print("="*90)
print("FINAL MODEL RANKING")
print("="*90)
print(final_comparison[['Model', 'Accuracy', 'Precision', 'Recall', 
                        'F1-Score', 'ROC-AUC', 'Mean CV Accuracy', 'Overall Score']].to_string(index=False))
print("="*90)

print(f"\n🏆 Best Model: {final_comparison.iloc[0]['Model']}")
print(f"   Accuracy: {final_comparison.iloc[0]['Accuracy']:.4f}")
print(f"   F1-Score: {final_comparison.iloc[0]['F1-Score']:.4f}")
print(f"   ROC-AUC: {final_comparison.iloc[0]['ROC-AUC']:.4f}")

plt.figure(figsize=(12, 7))
bars = plt.barh(final_comparison['Model'], final_comparison['Overall Score'], 
               color=plt.cm.viridis(np.linspace(0, 1, len(final_comparison))))

plt.xlabel('Overall Score', fontsize=12)
plt.title('Model Ranking by Overall Performance Score', fontsize=14, fontweight='bold')
plt.xlim(0, 1)

for bar, score in zip(bars, final_comparison['Overall Score']):
    plt.text(score + 0.01, bar.get_y() + bar.get_height()/2, 
            f'{score:.4f}', va='center', fontsize=10, fontweight='bold')

plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
accuracy_diffs = []
model_names = []

for idx in range(len(comparison_df)):
    if idx == 0:
        continue
    diff = comparison_df.iloc[0]['Accuracy'] - comparison_df.iloc[idx]['Accuracy']
    accuracy_diffs.append(diff)
    model_names.append(comparison_df.iloc[idx]['Model'])

plt.figure(figsize=(10, 6))
bars = plt.barh(model_names, accuracy_diffs, color='crimson', alpha=0.7)
plt.xlabel('Accuracy Difference from Best Model', fontsize=12)
plt.title('Accuracy Differences from Best Model', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)

for bar, val in zip(bars, accuracy_diffs):
    plt.text(val + 0.001 if val >= 0 else val - 0.001, 
             bar.get_y() + bar.get_height()/2, 
             f'{val:.4f}', va='center', fontsize=9, 
             ha='left' if val >= 0 else 'right')

plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAccuracy differences from {comparison_df.iloc[0]['Model']} (Best Model):")
for name, diff in zip(model_names, accuracy_diffs):
    print(f"  {name:25s}: {diff:.4f}")